Prueba tecnica


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          9997 non-null   object 
 1   name        9997 non-null   object 
 2   company_id  9996 non-null   object 
 3   amount      10000 non-null  float64
 4   status      10000 non-null  object 
 5   created_at  10000 non-null  object 
 6   paid_at     6009 non-null   object 
dtypes: float64(1), object(6)
memory usage: 547.0+ KB


Siguiendo el esquema 1.3 los siguientes valores no deben ser nulos


In [3]:
id_column = "id"
conpany_id = "company_id"
amount = "amount"
status = "status"
created_at = "created_at"


columns_to_check = ['id', 'company_id', 'amount', 'status', 'created_at']

null_check = df[columns_to_check].isna().any()

for column, has_nulls in null_check.items():
    print(f"{column} contains null values: {has_nulls}")

id contains null values: True
company_id contains null values: True
amount contains null values: False
status contains null values: False
created_at contains null values: False


Se eliminan las filas con ids nulos ya que estos valores son unicos, en este caso se puede hacer una investigacion mas detallada de los datos para no perder algun pago que no se registro correctamente.


In [4]:
df_clean = df.dropna(subset=[id_column, conpany_id])
null_check_clean = df_clean[columns_to_check].isna().any()
for column, has_nulls in null_check_clean.items():
    print(f"{column} contains null values: {has_nulls}")
    



id contains null values: False
company_id contains null values: False
amount contains null values: False
status contains null values: False
created_at contains null values: False


Se puede observar que nuestro dataset esta limpio en los campos que nos interesa, no fue necesario hacer alguna otra transformacion


In [5]:
df_clean.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 9993 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          9993 non-null   object 
 1   name        9990 non-null   object 
 2   company_id  9993 non-null   object 
 3   amount      9993 non-null   float64
 4   status      9993 non-null   object 
 5   created_at  9993 non-null   object 
 6   paid_at     6006 non-null   object 
dtypes: float64(1), object(6)
memory usage: 624.6+ KB


Transformando los datos vacios a tipo none (null) y asi no tener porblemas con la base de datos


Al crear la vista, noté que hay un ID que no coincide con el correcto, pero sí contiene el nombre de la compañía. Por este motivo, se realiza esta verificación y transformación de los datos.


Verificamos si hay inconsistencias en el ID y se encontró una. Este valor podría reemplazarse por la moda para obtener datos más limpios.


In [6]:
mode_ids = (
    df_clean
    .groupby('name')['company_id']
    .agg(lambda s: s.mode().iat[0])
)

df_clean.loc[:, 'canonical_id'] = df_clean['name'].map(mode_ids)

mismatched = df_clean[df_clean['company_id'] != df_clean['canonical_id']].drop(columns='canonical_id')

display(mismatched)

C:\Users\alvar\AppData\Local\Temp\ipykernel_53684\371180580.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.loc[:, 'canonical_id'] = df_clean['name'].map(mode_ids)


,id,name,company_id,amount,status,created_at,paid_at
603,701ed7d3e728c41ecf58639f64de12d7dc2fc4df,MiPasajefy,*******,92.66,paid,2019-02-27,2019-02-27
731,ec79a21ef969c7fc6beef080ff56baf0aeeca8b5,NaN,cbf1c8b09cd5b549416d49d220a40cbd317f952e,112.52,paid,2019-02-14,2019-02-14
2199,6a6ac16d53a02ba7948bff0a534e45404e716c5b,NaN,cbf1c8b09cd5b549416d49d220a40cbd317f952e,244.88,voided,2019-05-09,NaN
2200,4740cf2624c3b929d9944cdfcb5c87e71a82ddc5,NaN,cbf1c8b09cd5b549416d49d220a40cbd317f952e,118.78,paid,2019-03-05,2019-03-06


Remplazamos el valor del ID erroneo por la moda para no tener algun error


In [7]:
mode_ids = df_clean.groupby("name")["company_id"].agg(lambda x: x.mode()[0])
comon_id = mode_ids["MiPasajefy"]
mask = df_clean['company_id'].str.len() < 10

if mask.any():
    display(df_clean[mask])
    df_clean.loc[mask, "company_id"] = comon_id

    display(df_clean[mask])


,id,name,company_id,amount,status,created_at,paid_at,canonical_id
603,701ed7d3e728c41ecf58639f64de12d7dc2fc4df,MiPasajefy,*******,92.66,paid,2019-02-27,2019-02-27,cbf1c8b09cd5b549416d49d220a40cbd317f952e


,id,name,company_id,amount,status,created_at,paid_at,canonical_id
603,701ed7d3e728c41ecf58639f64de12d7dc2fc4df,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,92.66,paid,2019-02-27,2019-02-27,cbf1c8b09cd5b549416d49d220a40cbd317f952e


Guardamos el nuevo data set limpio


In [8]:
new_df = "data/clean_data.csv"
df_clean.to_csv(new_df, index=False)